In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('keystroke.csv')

X = data.iloc[:, 3:]
y = data['subject']

# divide data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest model
randomForestModel = RandomForestClassifier()

# Train the model
randomForestModel.fit(X_train, y_train)

# Evaluate the model on the test set
accuracy = randomForestModel.score(X_test, y_test)
print('Test accuracy:', accuracy)

Test accuracy: 0.9338559529642332


In [6]:
#accuracy of 0.72
# from sklearn.linear_model import LogisticRegression

# # Create a Logistic Regression model
# model = LogisticRegression(max_iter=2000)

# # Train the model
# model.fit(X_train, y_train)

# # Evaluate the model on the test set
# accuracy = model.score(X_test, y_test)
# print('Test accuracy:', accuracy)


In [16]:
#accuracy of 0.84
# from sklearn.preprocessing import StandardScaler

# # Scale the input features
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Create a Logistic Regression model and train it on the scaled data
# model = LogisticRegression(solver='sag', max_iter=5000)
# model.fit(X_train_scaled, y_train)

# # Evaluate the model on the scaled test set
# accuracy = model.score(X_test_scaled, y_test)
# print('Test accuracy:', accuracy)


Test accuracy: 0.8407643312101911


c:\Users\Korisnik\Desktop\Racunalna-forenzika-22-23\env\lib\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


In [12]:
#accuracy of 0.72
# from sklearn.tree import DecisionTreeClassifier

# # Create a Decision Tree model
# model = DecisionTreeClassifier()

# # Train the model
# model.fit(X_train, y_train)

# # Evaluate the model on the test set
# accuracy = model.score(X_test, y_test)
# print('Test accuracy:', accuracy)


Test accuracy: 0.7199902008819207


In [14]:
#accuracy of 0.79
# from sklearn.svm import SVC

# # Create an SVM model
# model = SVC()

# # Train the model
# model.fit(X_train, y_train)

# # Evaluate the model on the test set
# accuracy = model.score(X_test, y_test)
# print('Test accuracy:', accuracy)


Test accuracy: 0.7893189612934836


In [4]:
#accuracy of 0.74
# from sklearn.neighbors import KNeighborsClassifier

# # Create a KNN model
# model = KNeighborsClassifier()

# # Train the model
# model.fit(X_train, y_train)

# # Evaluate the model on the test set
# accuracy = model.score(X_test, y_test)
# print('Test accuracy:', accuracy)

Test accuracy: 0.7410583047525723
